# Import the necessary libraries

In [5]:
!pip install keras

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

### Load the data into Pandas dataframe

In [9]:
import numpy as np
train = open('data_for_train_and_test.txt', encoding='utf-8')

lines = train.readlines()
target = []
phrase = []

for line in lines:
    target.append(float(line[0]))
    phrase.append(line[2:])

target = np.array(target)
phrase = np.array(phrase)

In [11]:
X = phrase
Y = target
#Y = le.fit_transform(Y)
#Y = Y.reshape(-1,1)

Split into training and test data.

In [20]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

print(len(X_train))

5803


### Process the data
* Tokenize the data and convert the text to sequences.
* Add padding to ensure that all the sequences have the same shape.
* There are many ways of taking the *max_len* and here an arbitrary length of 150 is chosen.

In [13]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

### RNN
Define the RNN structure.

In [14]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

Call the function and compile the model.

In [15]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

Fit on the training data.

In [ ]:
model.fit(sequences_matrix, Y_train,batch_size=64, epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

The model performs well on the validation set and this configuration is chosen as the final model.

Process the test set data.

In [17]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

Evaluate the model on the test set.

In [18]:
accr = model.evaluate(test_sequences_matrix,Y_test)

33/33 [==============================] - 1s 21ms/step - loss: 0.0226 - accuracy: 0.9922


In [19]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.023
  Accuracy: 0.992


In [24]:
import pickle
with open('LSTM_trained_model.pkl', 'wb') as fout:
  pickle.dump((model), fout)

INFO:tensorflow:Assets written to: ram://d20facb6-c8ea-4d5f-a1f9-0e930bde103b/assets


INFO:tensorflow:Assets written to: ram://d20facb6-c8ea-4d5f-a1f9-0e930bde103b/assets


In [31]:
import pickle

with open('LSTM_trained_model.pkl', 'rb') as fin:
  clf = pickle.load(fin)

  X_new =["это мои любимык кроссовки"]
  test_sequences = tok.texts_to_sequences(X_new)
  test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

  res = np.array([1])
  a = model.evaluate(test_sequences_matrix, res)
  print(a)

1/1 [==============================] - 1s 504ms/step - loss: 0.8699 - accuracy: 0.0000e+00
[0.8699295520782471, 0.0]
